In [1]:
# entropy, mutual information, kl divergence, cross entropy, conditional entropy
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Tuple, Dict


In [2]:
# entropy definition based on a value-probability dictionary
def entropy(p:Dict[str, float]) -> float:
    return -sum([p[i]*math.log2(p[i]) for i in p])

example_alphabet_probs={'a':0.5, 'b':0.25, 'c':0.25}
print(entropy(example_alphabet_probs))

1.5


In [5]:
import heapq

# Define two example dictionaries with the same alphabet but different probabilities
example_probs_1 = {chr(97 + i): np.random.dirichlet(np.ones(26), size=1)[0][i] for i in range(26)}
example_probs_2 = {chr(97 + i): np.random.dirichlet(np.ones(26), size=1)[0][i] for i in range(26)}

# Calculate the entropy of the first dictionary
entropy_1 = entropy(example_probs_1)
print(f"Entropy of the first dictionary: {entropy_1}")

# Calculate the entropy of the second dictionary
entropy_2 = entropy(example_probs_2)
print(f"Entropy of the second dictionary: {entropy_2}")

# Calculate the mutual information
def mutual_information(p: Dict[str, float], q: Dict[str, float]) -> float:
    return sum([p[i] * math.log2(p[i] / q[i]) for i in p])

mutual_info = mutual_information(example_probs_1, example_probs_2)
print(f"Mutual Information: {mutual_info}")

# Create an optimal coding scheme for the first dictionary
def optimal_coding_scheme(p: Dict[str, float]) -> Dict[str, str]:
    sorted_probs = sorted(p.items(), key=lambda item: item[1], reverse=True)
    coding_scheme = {}
    code = 0
    for symbol, prob in sorted_probs:
        coding_scheme[symbol] = format(code, 'b').zfill(int(-math.log2(prob)))
        code += 1
    return coding_scheme

coding_scheme_1 = optimal_coding_scheme(example_probs_1)
print(f"Optimal Coding Scheme for the first dictionary: {coding_scheme_1}")
# Generate a bunch of values from the second dictionary
def generate_values(p: Dict[str, float], n: int) -> List[str]:
    symbols = list(p.keys())
    probabilities = list(p.values())
    return list(np.random.choice(symbols, size=n, p=probabilities))

generated_values = generate_values(example_probs_2, 1000)
print(f"Generated values from the second dictionary: {generated_values[:50]}")

# Demonstrate the difference in coding schemes
def encode_values(values: List[str], coding_scheme: Dict[str, str]) -> List[str]:
    return [coding_scheme[value] for value in values]

encoded_values_1 = encode_values(generated_values, coding_scheme_1)
print(f"Encoded values using the first dictionary's coding scheme: {encoded_values_1[:50]}")

# Create an optimal coding scheme for the second dictionary (Huffman coding)

class HuffmanNode:
    def __init__(self, symbol, prob):
        self.symbol = symbol
        self.prob = prob
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.prob < other.prob

def huffman_coding(p: Dict[str, float]) -> Dict[str, str]:
    heap = [HuffmanNode(symbol, prob) for symbol, prob in p.items()]
    heapq.heapify(heap)
    
    while len(heap) > 1:
        node1 = heapq.heappop(heap)
        node2 = heapq.heappop(heap)
        merged = HuffmanNode(None, node1.prob + node2.prob)
        merged.left = node1
        merged.right = node2
        heapq.heappush(heap, merged)
    
    root = heap[0]
    coding_scheme = {}
    
    def generate_codes(node, code):
        if node.symbol is not None:
            coding_scheme[node.symbol] = code
        else:
            generate_codes(node.left, code + '0')
            generate_codes(node.right, code + '1')
    
    generate_codes(root, '')
    return coding_scheme

coding_scheme_2 = huffman_coding(example_probs_2)
print(f"Optimal Coding Scheme for the second dictionary (Huffman): {coding_scheme_2}")

encoded_values_2 = encode_values(generated_values, coding_scheme_2)
print(f"Encoded values using the second dictionary's coding scheme: {encoded_values_2[:50]}")

Entropy of the first dictionary: 2.968237770572419
Entropy of the second dictionary: 3.3371706097649607
Mutual Information: 0.607478068353195
Optimal Coding Scheme for the first dictionary: {'w': '00', 'l': '0001', 'e': '0010', 'a': '0011', 's': '0100', 'i': '0101', 'f': '0110', 'u': '0111', 'm': '1000', 'q': '01001', 'o': '01010', 'j': '01011', 'x': '001100', 't': '001101', 'n': '001110', 'r': '001111', 'z': '010000', 'h': '0010001', 'g': '0010010', 'p': '0010011', 'c': '0010100', 'd': '00010101', 'y': '00010110', 'k': '000010111', 'v': '000011000', 'b': '00000011001'}


ValueError: probabilities do not sum to 1